In [1]:
import pandas as pd
import os
import networkx as nx
import community
import numpy as np
import pickle

from tqdm import tqdm
import copy
import sys
import operator
sys.path.insert(0, '/home/vincent/Documents/School/Research/EoBoR/')
from hungarian import Hungarian

In [2]:
nodestats_folder = "nodestats_folder/"

In [3]:
def transposeDict(original_dict,items=False):
    from collections import defaultdict
    result = defaultdict(list)
    items_l = original_dict.items() if not items else original_dict
    for k,v in items_l:
        result[v].append(k)
    result = dict(result)
    return result

# Labelling dynamic communities

Two methods:
* Hungarian method with cutoff
* Santo's "best match"

##### Pulling networks from data

In [ ]:
partitions_over_time = {}

for filename in sorted(os.listdir(nodestats_folder)):
    with open(nodestats_folder+filename,'rb') as f:
        df = pickle.load(f)
        for index,row in df.iterrows():
            partitions_over_time[row['username']] = row['Community']

partitions_over_time = transposeDict(partitions_over_time)

##### Generating test networks

In [4]:
import community

In [5]:
networks_over_time = []
partitions_over_time = []

networks_over_time.append(nx.random_geometric_graph(500,0.125))
partitions_over_time.append(transposeDict(dict(community.best_partition(networks_over_time[0]))))

for x in tqdm(range(100)):
    g = nx.Graph(networks_over_time[x])
    #randomly delete a hundred nodes
    removed_nodes = []
    for i in range(100):
        removed_node = np.random.choice(list(g.nodes()))
        removed_nodes.append(removed_node)
        g.remove_node(removed_node)
    for i in list(nx.isolates(g)):
        g.remove_node(i)
        removed_nodes.append(i)
    #randomly add nodes by neighbors
    for node in removed_nodes:
        if nx.number_of_nodes(g)==0:
            g.add_node(node)
        elif nx.number_of_nodes(g)==1:
            g.add_edge(list(g.nodes())[0],node)
        else:
            neighbors = []
            while neighbors==[]:
                select_node = np.random.choice(list(g.nodes()))
                neighbors = list(g.neighbors(select_node))
            neighbors_deg = [j for q,j in list(g.degree(neighbors))]
            neighbors_deg = [j/sum(neighbors_deg) for j in neighbors_deg]
            select_neighbor = np.random.choice(neighbors,p=neighbors_deg)
            g.add_edge(select_neighbor,node)
    networks_over_time.append(g)
    partitions_over_time.append(transposeDict(dict(community.best_partition(g))))

100%|██████████| 100/100 [00:03<00:00, 25.56it/s]


## Hungarian method

#### Generating reassignments

* If there is a forward map and it matches the backward map, then it's good. 
* If there is a forward map but a different backward map, it means a forward handoff (comm1 becomes comm2)
* If there is a backward map but a different forward map, same kind of handoff but different specific handoff. 
* If there is no backward map, the community dies. 
* If there is no forward map, the community is created. 

In [ ]:
def generateReassignments(partitions_over_time):
    reassignment_partitions = []

    prev_part = partitions_over_time[0]#Start with the first one in the time series
    reassignment_partitions.append({x:x for x in prev_part.keys()}) #Initialize a 1-1 mapping for first part
    for next_part in tqdm(partitions_over_time[1:]):
        #Computer matrix of Jaccard distances between communities
        jacmatrix = np.zeros((len(prev_part),len(next_part)))
        for x in prev_part.keys():
            for y in next_part.keys():
                one = set(prev_part[x])
                two = set(next_part[y])
                jacmatrix[int(x)][int(y)] = 1 - len(one&two)/len(one|two)  ### DOES THIS NEED A THRESHOLD???
        #Hungarian from prev to next (forward)
        hungarian = Hungarian(jacmatrix)
        hungarian.calculate()
        forward_map = hungarian.get_results()
        
        #TEST SCENARIO
        valid_partitions = forward_map
        #Create a reassignment dictionary for next_part
        this_assignment_dict = {y:x for x,y in valid_partitions} #Formatted as "this_key:will_become_this_val"

        reassignment_partitions.append(this_assignment_dict)
        prev_part = next_part
    return reassignment_partitions

#### Confirming reassignments

At this point, `reassignment_partitions` is a list of dictionaries that tell you how to relabel communities to the corresponding label of the previous time step. 

The following generates `final_partitions`, a list of dictionaries that tell you how to relabel communities to an absolute label over time. 

In [ ]:
def finalizingPartitions(reassignment_partitions):
    final_partitions = []
    final_partitions.append(reassignment_partitions[0])
    prev_dict = reassignment_partitions[0]
    new_comm_label = max([int(x) for x in prev_dict.keys()]) + 1

    #Go through the reassignments
    for x in range(len(reassignment_partitions[1:])):
        current_dict = copy.deepcopy(reassignment_partitions[x])
        for k,v in current_dict.items():
            if v in prev_dict.keys():
                current_dict[k] = prev_dict[v]
            elif v not in prev_dict.keys():
                current_dict[k] = new_comm_label
                new_comm_label+=1
        final_partitions.append(current_dict)
        prev_dict = current_dict
    return final_partitions

## Testing example networks with Hungarian

8/6/18 -- There are issues where Hungarian can't do a forward and backward map so it can't sense when new communities are created. Also, because the mapping forces a 1-to-1 relationship, there's an issue with overzealous fitting. 

In [ ]:
reassignment_partitions = generateReassignments(partitions_over_time)

In [ ]:
final_partitions = finalizingPartitions(reassignment_partitions)

In [ ]:
final_partitions

In [ ]:
converted_partitions_over_time = []

for x,part in enumerate(partitions_over_time):
    print(x)
    result = {final_partitions[x][k]:part[k] for k in part}
    converted_partitions_over_time.append( result )

# Santo reassignment

### Design conceits

By keeping track of forward and backward maximum matches by Jaccard distance, this opens the possibility for a lot of different ways for communities to merge in and one of each other. 

It's relatively clear that if two communities has a best forward match with the same subsequent community, a merge has occurred. It is also relatively clear that if two communities draw from the same previous community, a split has occurred. 

It's not clear how to label communities that exchange members. So for the sake of this work, I'm just going to say that we're keeping track of splits but not merges. However, this opens up the possibility for a ship of theseus problem with a community that routinely exchanges members. 

In [10]:
def santoReassignments(partitions_over_time, tau = 0.05):
    reassignment_partitions = []
    
    prev_part = partitions_over_time[0] #Start with the first one in the time series
    reassignment_partitions.append({x:x for x in prev_part.keys()}) #Initialize a 1-1 mapping for first part
    max_k = max(list(prev_part.keys()))

    for i in tqdm(range(1,len(partitions_over_time))):
        next_part = partitions_over_time[i]
        jacmatrix = {}
        for x in prev_part.keys():
            if x not in jacmatrix:
                jacmatrix[x] = {}
            for y in next_part.keys():
                one = set(prev_part[x])
                two = set(next_part[y])
                temp = len(one&two)/len(one|two)
                jacmatrix[int(x)][int(y)] = temp if temp>tau else 0
        jacmatrix_t = {}
        for bigkey in jacmatrix:
            for smallkey in jacmatrix[bigkey]:
                if smallkey not in jacmatrix_t:
                    jacmatrix_t[smallkey] = {}
                if bigkey not in jacmatrix_t[smallkey]:
                    jacmatrix_t[smallkey][bigkey] = jacmatrix[bigkey][smallkey]
        
        forward_mapping = {}
        backward_mapping = {}
        for bigkey in jacmatrix:
            forward_mapping[bigkey] = max(jacmatrix[bigkey].items(), key=operator.itemgetter(1))[0]
        for bigkey in jacmatrix_t:
            backward_mapping[bigkey] = max(jacmatrix_t[bigkey].items(), key=operator.itemgetter(1))[0]
                
        old_map = reassignment_partitions[-1]
        new_map = {}
        #new_relabels = {}
        #If something says it tracks forward, then track it
        forward_mapping_t = transposeDict(forward_mapping)
        for k_1,v_0 in forward_mapping_t.items():
            if len(v_0)==1: #One-to-one forward relationship
                new_map[v_0[0]] = k_1
            elif len(v_0)>1: #One-to-many forward relationship
                max_v = -1
                max_v_index = -1
                for thing in v_0: #Get the largest similarity score
                    if jacmatrix[thing][k_1]>max_v:
                        max_v = jacmatrix[thing][k_1]
                        max_v_index = thing
                new_map[max_v_index] = k_1
                #Record the rest as being possible MERGES
#                 for thing in [jk for jk in v_0 if jk!=max_v_index]:
#                     problem_mappings[thing] = k_1
        backward_mapping_t = transposeDict(backward_mapping)
        for k_0,v_1 in backward_mapping_t.items():
            if len(v_1)==1 and k_0 in new_map and new_map[k_0]==v_1[0]:
                pass #Confirm sustain
            elif len(v_1)>1:
                for thing in v_1:
                    if k_0 in new_map and new_map[k_0]==thing:
                        pass #Confirm it's already being written in, so a sustain
                    elif k_0 in new_map and new_map[k_0]!=thing:
                        #Confirm a split
                        new_map[max_k] = thing
                        max_k += 1
                        #Don't actually have to keep track of mappings to newly relabeled communities
                    elif k_0 not in new_map:
                        new_map[k_0] = thing #Only a backward relationship
        #Reverse new_map so that it becomes a dictionary of relabels
        #Filter new_map through old_map
        new_names = {v:k if k not in old_map else old_map[k] for k,v in new_map.items()}
        
        reassignment_partitions.append(new_names)
        prev_part = next_part
    
    return reassignment_partitions

## Testing example networks with Santo

8/6/18 -- Need to add in the ability to do both forward and backward mapping. Draft of this has been started above. 

In [11]:
santo_reassignment_partitions = santoReassignments(partitions_over_time)

100%|██████████| 100/100 [00:00<00:00, 1508.52it/s]


In [13]:
santo_reassignment_partitions

[{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7},
 {0: 0, 1: 1, 2: 2, 3: 6, 4: 7, 5: 4, 6: 3, 7: 5},
 {0: 0, 1: 1, 2: 2, 3: 6, 4: 7, 5: 4, 6: 3, 7: 5, 8: 8},
 {0: 0, 1: 2, 2: 6, 3: 4, 4: 3, 5: 1, 6: 8, 7: 7},
 {0: 0, 1: 2, 2: 8, 3: 4, 4: 3, 5: 1, 6: 6, 7: 8, 8: 7},
 {0: 0, 1: 2, 2: 3, 3: 8, 4: 4, 5: 1, 6: 6, 7: 9, 8: 8, 9: 10, 10: 7},
 {0: 8,
  1: 2,
  2: 14,
  3: 8,
  4: 9,
  5: 4,
  6: 3,
  7: 1,
  8: 6,
  9: 0,
  10: 10,
  11: 16,
  12: 11,
  13: 17,
  14: 15,
  15: 12,
  16: 13},
 {0: 8,
  1: 0,
  2: 9,
  3: 4,
  4: 14,
  5: 3,
  6: 1,
  7: 6,
  8: 10,
  9: 16,
  10: 20,
  11: 18,
  12: 19,
  13: 15,
  14: 17,
  15: 8,
  16: 12,
  17: 13},
 {0: 8,
  1: 0,
  2: 9,
  3: 21,
  4: 22,
  5: 14,
  6: 3,
  7: 16,
  8: 1,
  9: 23,
  10: 18,
  11: 6,
  12: 13,
  13: 19,
  15: 20,
  16: 17,
  17: 8,
  18: 12},
 {0: 8,
  1: 22,
  2: 21,
  3: 9,
  4: 26,
  5: 24,
  6: 3,
  7: 16,
  8: 1,
  9: 23,
  10: 14,
  11: 20,
  12: 27,
  13: 6,
  14: 14,
  15: 0,
  16: 28,
  17: 19,
  18: 8,
  19: 17,


In [12]:
santo_final_partitions = TRInalizingPartitions(santo_reassignment_partitions)

NameError: name 'TRInalizingPartitions' is not defined

## Exporting results to original dataframes

In [ ]:
#NEED STUFF HERE

# Testing

In [ ]:
a = {x:np.random.randint(0,5) for x in range(20)}

In [ ]:
transposeDict(a)

In [ ]:

        sustain = {}
        forward_pass = {}
        backward_pass = {}
        for k_f,v_f in forward_mapping.items():
            for k_b,v_b in backward_mapping.items():
                if k_f==v_b and v_f==k_b:
                    sustain[k_f] = v_f
                

In [ ]:
#Not sure if I'll still need this code

def TRInalizingPartitions(reassignment_partitions):
    final_partitions = []
    final_partitions.append(reassignment_partitions[0])
    prev_dict = reassignment_partitions[0]
    new_comm_label = max([int(x) for x in prev_dict.keys()]) + 1

    #Here, reassignment_partitions == (sustain, forward_pass, backward_pass)
    for x in tqdm(range(1,len(reassignment_partitions[1:]))):
        current_sustain = copy.deepcopy(reassignment_partitions[x][0])
        current_forward = copy.deepcopy(reassignment_partitions[x][1])
        current_backward = copy.deepcopy(reassignment_partitions[x][2])
        current_dict = {}
        accommodated_keys = set()
        print(reassignment_partitions[x])
        print(current_sustain)
        for k,v in current_sustain.items():
            if v in prev_dict.keys():
                current_dict[k] = prev_dict[v]
                accommodated_keys.add(k)
            #else: actually this is okay because it means it exists in forward or backward passes
                #raise ValueError("Something went wrong. A sustain was unable to locate its originating source.")
        for k,v in current_forward.items():
            accommodated_keys.add(k)
        for k,v in current_backward.items():
            current_dict[k] = new_comm_label
            new_comm_label += 1
        if set(list(prev_dict.keys()))!=accommodated_keys:
            print(list(prev_dict.keys()))
            print(accommodated_keys)
            raise ValueError("Missing something")
        final_partitions.append(current_dict)
    return final_partitions

In [ ]:
        for k,v in backward_mapping.items():
            if v in new_map and new_map[v]==k:
                pass #CONFIRMED SUSTAINS
            elif v in new_map and new_map[v]!=k:
                #FORWARD PASS
            else:
                [DO SOMETHING ABOUT THE NEW COMMUNITIES]
        #Keep track of renamings?
        #Double entry book-keeping. 
            #Make sure that all your sustains, births, and deaths add up to the pre- and post-community counts
        


In [ ]:
def santoReassignments(partitions_over_time):
    reassignment_partitions = []
    
    prev_part = partitions_over_time[0] #Start with the first one in the time series
    reassignment_partitions.append({x:x for x in prev_part.keys()}) #Initialize a 1-1 mapping for first part
    max_k = max(list(prev_part.keys()))
    
    for i in tqdm(range(1,len(partitions_over_time[1:]))):
        next_part = partitions_over_time[i]
        jacmatrix = np.zeros((len(prev_part),len(next_part)))
        for x in prev_part.keys():
            for y in next_part.keys():
                one = set(prev_part[x])
                two = set(next_part[y])
                jacmatrix[int(x)][int(y)] = 1 - len(one&two)/len(one|two)
        forward_mapping = {}
        for x in range(len(jacmatrix)):
            forward_mapping[x] = np.argmin(jacmatrix[x][:])
        jacmatrix_t = np.transpose(jacmatrix)
        backward_mapping = {}
        for x in range(len(jacmatrix_t)):
            backward_mapping[x] = np.argmin(jacmatrix_t[x][:])
        
        #Sustain: If forward mapping equals backward mapping
        sustains = {x:forward_mapping[x] for x in forward_mapping if forward_mapping[x] in backward_mapping and backward_mapping[forward_mapping[x]]==x} 
        #Forward pass: Ends current community
        forward_pass = {x:forward_mapping[x] for x in forward_mapping if forward_mapping[x] in backward_mapping and backward_mapping[forward_mapping[x]]!=x}
        #Backward pass: Creates new community
        backward_pass = {x:backward_mapping[x] for x in backward_mapping if backward_mapping[x] in forward_mapping and forward_mapping[backward_mapping[x]]!=x}
        #SOMETHING FUNKY WITH BACKWARD PASSES
        
        old_map = reassignment_partitions[-1]
        new_map = {}
        print("Old map", old_map)
        print("Sustains", sustains)
        print("Forward passes", forward_pass)
        print("Backward passes", backward_pass)
        for k,v in sustains.items():
            new_map[k] = old_map[v]
        #Ignore forward passes
        for k,v in backward_pass.items():
            #new_k = max_k if k<=max_k else k #If the comm already existed in a previous step, make a new one
            #max_k += 1
            new_k = k
            new_map[new_k] = old_map[v]
        print("New map", new_map)
        reassignment_partitions.append(new_map)
        
        #Update the partition jaccard comparison thingy
        prev_part = next_part
        
    return reassignment_partitions

In [ ]:
        for x in range(len(prev_part)): #REWRITE 
            if np.maximum(jacmatrix[x][:]) != 0:
                forward_mapping[x] = np.argmax(jacmatrix[x][:])
        jacmatrix_t = np.transpose(jacmatrix)
        for x in range(len(next_part)):
            if np.maximum(jacmatrix_t[x][:]) != 0:
                backward_mapping[x] = np.argmax(jacmatrix_t[x][:])
        
